In [ ]:
#default_exp convert

In [ ]:
#export
from fastai.vision import *
from CinemaNet.train_utils import *
from CinemaNet.custom_head import *

### Load Model

In [ ]:
data = get_data('/home/rahul/github_projects/CinemaNet/', img_size=224, batch_size=1)

In [ ]:
learn = cnn_learner(data, models.mobilenet_v2)

## PyTorch ⟶ ONNX ⟶ CoreML

### PyTorch ⟶ ONNX

In [ ]:
fname = 'MNetV2-100x177-3000L'
learn.load(fname);

In [ ]:
body = learn.model[0]
head = learn.model[1]
head

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d(2560, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=2560, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=6, bias=True)
)

In [ ]:
#export
def torch_to_onnx(model:nn.Module,
                  save_path   :str   = '../exported-models/',
                  model_fname :str   = 'onnx-model',
                  input_shape :tuple = (3, 224, 224)):
    'Export torch.nn model as ONNX model'
    model.eval();
    x = torch.randn(1, *input_shape)
    torch.onnx._export(model, x.cuda(), 
                       f'{os.path.join(save_path, model_fname)}.onnx',
                       export_params=True)


In [ ]:
torch_to_onnx(learn.model, '/home/rahul/Desktop/', 'tmp_body')
torch_to_onnx(head, '/home/rahul/Desktop/', 'tmp_head', input_shape=(1280,7,7), )

### ONNX ⟶ CoreML (Raw)

In [ ]:
#export
import copy
import coremltools
import os

from onnx_coreml import convert
from onnx import onnx_pb

In [ ]:
imagenet_stats

([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [ ]:
#export
args = dict(is_bgr=False,
            red_bias   = -(0.485 * 255.0),
            green_bias = -(0.456 * 255.0),
            blue_bias  = -(0.406 * 255.0))

def onnx_to_coreml(model_path:str = '../exported-models/',
                   model_name:str = '',
                   num_labels:int = 6,
                   mode:str = 'classifier',
                   preprocessing_args:dict = args,
                   target_ios:str = '13'):
    'Convert ONNX to CoreML w/out scaling layer'
    return convert(
        #model=path_to_model, mode=mode, image_input_names=['Input-Image'], 
        model=f'{os.path.join(model_path, model_name)}.onnx',
        mode=mode,
        image_input_names=['input.1'], 
        class_labels=[i for i in range(num_labels)], preprocessing_args=args,
        target_ios=target_ios
    )

In [ ]:
#export
def add_scaler(model:coremltools.models.model.MLModel,
               author:str  = 'Rahul Somani / Synopsis',
               license:str = 'BSD',
               description:str = None) -> coremltools.proto.Model_pb2.Model:
    'Add scaling layer to model converted from ONNX using `onnx_coreml.convert`, and some metadata'
    spec    = model.get_spec()
    nn_spec = spec.neuralNetworkClassifier
    layers  = nn_spec.layers                 # list of all layers
    layers_copy = copy.deepcopy(layers)
    del nn_spec.layers[:]
    
    scale_layer = nn_spec.layers.add()
    scale_layer.name = 'scaler'
    scale_layer.input.append('input.1')
    scale_layer.output.append('input.1_scaled')
    
    params = scale_layer.scale
    scale_R = 1.0 / (0.229 * 255.0)
    scale_G = 1.0 / (0.224 * 255.0)
    scale_B = 1.0 / (0.225 * 255.0)
    params.scale.floatValue.extend([scale_R, scale_G, scale_B])
    params.shapeScale.extend([3,1,1])
    
    nn_spec.layers.extend(layers_copy) # add back the layers
    nn_spec.layers[1].input[0] = 'input.1_scaled'
    
    spec.description.metadata.author  = author
    spec.description.metadata.license = license
    
    if not description:
        from datetime import datetime
        time_right_now = datetime.now().strftime('%Y-%m-%d at %H:%M:%S')
        spec.description.metadata.shortDescription = f'''This model was exported to CoreML on {time_right_now}
        Visit https://github.com/Synopsis/CinemaNet/tree/vision_models for more details 
        '''
    else:
        spec.description.metadata.shortDescription = description
    
    return spec

In [ ]:
#export
def save_coreml_model(model:coremltools.proto.Model_pb2.Model,
                      save_path:str  = '../models/',
                      model_name:str = 'coreml-model'):
    'Save model w/ scaling layer created using `add_scale`'
    coreml_model      = coremltools.models.MLModel(model)
    coreml_model_path = f'{os.path.join(save_path, model_name)}.mlmodel'
    coreml_model.save(coreml_model_path)

In [ ]:
mod_coreml_raw    = onnx_to_coreml(model_name=fname, num_labels=6, target_ios='13')
mod_coreml_scaler = add_scaler(model=mod_coreml_raw)
save_coreml_model(mod_coreml_scaler, model_name=fname)

1/162: Converting Node Type Conv
2/162: Converting Node Type BatchNormalization
3/162: Converting Node Type Clip
4/162: Converting Node Type Conv
5/162: Converting Node Type BatchNormalization
6/162: Converting Node Type Clip
7/162: Converting Node Type Conv
8/162: Converting Node Type BatchNormalization
9/162: Converting Node Type Conv
10/162: Converting Node Type BatchNormalization
11/162: Converting Node Type Clip
12/162: Converting Node Type Conv
13/162: Converting Node Type BatchNormalization
14/162: Converting Node Type Clip
15/162: Converting Node Type Conv
16/162: Converting Node Type BatchNormalization
17/162: Converting Node Type Conv
18/162: Converting Node Type BatchNormalization
19/162: Converting Node Type Clip
20/162: Converting Node Type Conv
21/162: Converting Node Type BatchNormalization
22/162: Converting Node Type Clip
23/162: Converting Node Type Conv
24/162: Converting Node Type BatchNormalization
25/162: Converting Node Type Add
26/162: Converting Node Type Conv


#### behind the scenes...

In [ ]:
spec = mod_converted.get_spec()
nn_spec = spec.neuralNetworkClassifier
layers = nn_spec.layers

In [ ]:
layers[0]

name: "327"
input: "input.1"
output: "327"
convolution {
  outputChannels: 32
  kernelChannels: 3
  nGroups: 1
  kernelSize: 3
  kernelSize: 3
  stride: 2
  stride: 2
  dilationFactor: 1
  dilationFactor: 1
  valid {
    paddingAmounts {
      borderAmounts {
        startEdgeSize: 1
        endEdgeSize: 1
      }
      borderAmounts {
        startEdgeSize: 1
        endEdgeSize: 1
      }
    }
  }
  weights {
    floatValue: 0.01318359375
    floatValue: -0.004322052001953125
    floatValue: 0.01482391357421875
    floatValue: 0.03277587890625
    floatValue: -0.025390625
    floatValue: 0.00685882568359375
    floatValue: 0.01055145263671875
    floatValue: -0.037353515625
    floatValue: -0.0147247314453125
    floatValue: 0.00798797607421875
    floatValue: -0.00591278076171875
    floatValue: 0.01507568359375
    floatValue: 0.0200042724609375
    floatValue: -0.032867431640625
    floatValue: -0.002086639404296875
    floatValue: 0.0113525390625
    floatValue: -0.032958984375


In [ ]:
layers[-1]

name: "488"
input: "487"
output: "488"
batchedMatmul {
  weightMatrixFirstDimension: 512
  weightMatrixSecondDimension: 6
  hasBias: true
  weights {
    floatValue: -0.00395965576171875
    floatValue: -0.06640625
    floatValue: 0.0295257568359375
    floatValue: -0.0186920166015625
    floatValue: 0.04339599609375
    floatValue: 0.03961181640625
    floatValue: -0.073486328125
    floatValue: 0.050201416015625
    floatValue: 0.0631103515625
    floatValue: -0.016357421875
    floatValue: 0.07421875
    floatValue: 0.045440673828125
    floatValue: 0.03302001953125
    floatValue: -0.0421142578125
    floatValue: 0.00606536865234375
    floatValue: -0.05157470703125
    floatValue: 0.03619384765625
    floatValue: 0.0306243896484375
    floatValue: -0.00496673583984375
    floatValue: 0.033203125
    floatValue: 0.0261688232421875
    floatValue: 0.01345062255859375
    floatValue: 0.024749755859375
    floatValue: -0.059814453125
    floatValue: -0.04656982421875
    floatValue: 0

In [ ]:
layers_copy = copy.deepcopy(layers)
del nn_spec.layers[:]

In [ ]:
scale_layer = nn_spec.layers.add()
scale_layer.name = 'scale_layer'
scale_layer.input.append('input.1')
scale_layer.output.append('input.1_scaled')
params = scale_layer.scale
params.scale.floatValue.extend([red_scale, green_scale, blue_scale])
params.shapeScale.extend([3, 1, 1])

nn_spec.layers.extend(layers_copy)
nn_spec.layers[1].input[0] = 'input.1_scaled'

In [ ]:
x = nn_spec.layers[0]
x

name: "scale_layer"
input: "input.1"
output: "input.1_scaled"
scale {
  shapeScale: 3
  shapeScale: 1
  shapeScale: 1
  scale {
    floatValue: 0.017124753445386887
    floatValue: 0.017507003620266914
    floatValue: 0.01742919348180294
  }
}

In [ ]:
path_to_model

'../models/MobileNetV2-img_224-frozen-opt_accuracy.onnx'

In [ ]:
fname

'MobileNetV2-img_224-frozen-opt_accuracy'

In [ ]:
spec.description.metadata.author = 'Rahul Somani / Synopsis'
spec.description.metadata.license = 'BSD'

from datetime import datetime
time_right_now = datetime.now().strftime('%Y-%m-%d at %H:%M:%S')
spec.description.metadata.shortDescription = f'''This model was exported to CoreML on {time_right_now}
Visit https://github.com/Synopsis/CinemaNet/tree/vision_models for more details 
'''

In [ ]:
nn_spec.layers[0]

name: "scale_layer"
input: "input.1"
output: "input.1_scaled"
scale {
  shapeScale: 3
  shapeScale: 1
  shapeScale: 1
  scale {
    floatValue: 0.017124753445386887
    floatValue: 0.017507003620266914
    floatValue: 0.01742919348180294
  }
}

In [ ]:
nn_spec.preprocessing

[]

In [ ]:
coreml_model = coremltools.models.MLModel(spec)
coreml_path = '../models/MobileNetV2-img_224-frozen-opt_accuracy.mlmodel'

In [ ]:
coreml_model.save(coreml_path)

## Big Fat `torch_to_coreml` Wrapper (for ImageNet end-to-end models only)

You should only use this function to convert models pretrained on ImageNet, as internally, that's the preprocessing applied

In [ ]:
#export
def torch_to_coreml(model:nn.Module,
                    num_labels:int     = 6,
                    save_path:str      = '../exported-models/',
                    model_fname:str    = 'best-model',
                    mode:str           = 'classifier',
                    input_shape:tuple  = (3,224,224),
                    author:str         = 'Rahul Somani / Synopsis',
                    license:str        = 'BSD', # 
                    description:str    = None):
    'Export torch.nn model to CoreML model'
    torch_to_onnx(model, model_fname=model_fname, input_shape=input_shape) # exports to disk
    mod_coreml_raw    = onnx_to_coreml(save_path, model_fname, num_labels, mode, target_ios='13')
    mod_coreml_scaler = add_scaler(mod_coreml_raw, author, license, description)
    save_coreml_model(mod_coreml_scaler, save_path, model_fname)
    print(f'Saved model to {os.path.join(save_path, model_fname)}.mlmodel')

In [ ]:
model_fname = 'MNetV2-100x177-3000L'

In [ ]:
torch_to_coreml(learn.model, learn.data.c, '/home/rahul/Desktop/', 'tmp_body')

1/162: Converting Node Type Conv
2/162: Converting Node Type BatchNormalization
3/162: Converting Node Type Clip
4/162: Converting Node Type Conv
5/162: Converting Node Type BatchNormalization
6/162: Converting Node Type Clip
7/162: Converting Node Type Conv
8/162: Converting Node Type BatchNormalization
9/162: Converting Node Type Conv
10/162: Converting Node Type BatchNormalization
11/162: Converting Node Type Clip
12/162: Converting Node Type Conv
13/162: Converting Node Type BatchNormalization
14/162: Converting Node Type Clip
15/162: Converting Node Type Conv
16/162: Converting Node Type BatchNormalization
17/162: Converting Node Type Conv
18/162: Converting Node Type BatchNormalization
19/162: Converting Node Type Clip
20/162: Converting Node Type Conv
21/162: Converting Node Type BatchNormalization
22/162: Converting Node Type Clip
23/162: Converting Node Type Conv
24/162: Converting Node Type BatchNormalization
25/162: Converting Node Type Add
26/162: Converting Node Type Conv


### Export

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_fastai_custom_head.ipynb.
Converted 02_wandb_callback.ipynb.
Converted 03_pytorch_to_coreml.ipynb.
Converted 04_mobilenet_v2_basic.ipynb.
Converted 05_mixmatch-mobilenetv2-600L.ipynb.
Converted 05_mixmatch-wrn-custom-from_scratch.ipynb.
Converted 06_mixmatch-mobilenet-3000L.ipynb.
Converted Untitled.ipynb.
Converted exploring-architectures.ipynb.
Converted mobilenetV3-large1.0-3000L.ipynb.
